In [18]:
from dotenv import load_dotenv

load_dotenv()

True

In [19]:
import os

print(os.getenv("NVIDIA_API_KEY"))

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


In [20]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

model = ChatNVIDIA(model="meta/llama-3.1-405b-instruct")

In [21]:
from langchain_community.tools import TavilySearchResults

search = TavilySearchResults(max_results=2)

In [22]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = Chroma.from_documents(documents, NVIDIAEmbeddings(model_name="nvidia/llama-3.2-nv-embedqa-1b-v2"))
retriever = vector.as_retriever()

In [23]:
from langchain_core.tools import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [24]:
tools = [search, retriever_tool]

In [25]:
model_with_tools = model.bind_tools(tools)

In [ ]:
from langchain import hub

prompt = hub.pull("hwchase17/react-chat")

In [28]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react-chat', 'lc_hub_commit_hash': '3ecd5f710db438a9cf3773c57d6ac8951eefd2cd9a9b2a0026a65a0893b86a6e'}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative res

In [29]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt)

In [48]:
print(agent.steps[1].template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

In [49]:
agent.invoke(
    {
        "input": "Who is Japan's prime minister?",
        "intermediate_steps": [],
        "chat_history": [],
    }
)

KeyError: "Input to PromptTemplate is missing variables {'tools', 'tool_names'}.  Expected: ['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'] Received: ['input', 'intermediate_steps', 'chat_history', 'agent_scratchpad']\nNote: if you intended {tools} to be part of the string and not a variable, please escape it with double curly braces like: '{{tools}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "